# Introduction

Multi-instance (MI) machine learning approaches can be used to solve the issues of representation of each molecule by multiple conformations (instances) and automatic selection of the most relevant ones. In the multi-instance approach, an example (i.e., a molecule) is presented by a bag of instances (i.e., a set of conformations), and a label (a molecule property value) is available only for a bag (a molecule), but not for individual instances (conformations).

In this study, we have implemented several multi-instance algorithms, both conventional and based on deep learning, and investigated their performance. We have compared the performance of MI-QSAR models with those based on the classical single-instance QSAR (SI-QSAR) approach in which each molecule is encoded by either 2D descriptors computed for the corresponding molecular graph or 3D descriptors issued for a single lowest-energy conformation. 

<img src="img/toc.png" width="600"/>

# Descriptors

Сonformations representing each molecule were generated using the algorithm implemented in RDKit. In our study, we generated up to 100 conformations and removed conformations with RMSD values below 0.5Å to the remaining ones to reduce redundancy.For the descriptor representation of conformations, we used previously developed 3D pharmacophore signatures. Each conformation is represented by a set of pharmacophore features (H-bond donor/acceptor, the center of positive/negative charge, hydrophobic, and aromatic) determined by applying the corresponding SMARTS patterns. All possible quadruplets of features of a particular conformation were enumerated. Distances between features were binned to allow fuzzy matching of quadruplets with small differences in the position of features. Here we used the 1Å bin step as it demonstrated reasonable performance in our previous studies. These signatures consider distances between features and their spatial arrangement to recognize the stereo configuration of quadruplets. We counted the number of identical 3D pharmacophore quadruplet signatures for each conformation and used the obtained vectors as descriptors for model building. 3D pharmacophore descriptors used in this study were implemented in the pmapper Python package (https://github.com/DrrDom/pmapper). 
To build 2D models, we chose binary Morgan fingerprints (MorganFP) of radius 2 and size 2048 calculated with RDKit because they are widely used 2D descriptors. For comparative purpose we also used 2D physicochemical descriptors (PhysChem) and binary 2D pharmacophore fingerprints (PharmFP) calculated with RDKit.

Calculate descriptors for multiple conformations:

In [ ]:
import os
from miqsar.utils import calc_3d_pmapper

#Choose dataset to be modeled and create a folder where the descriptors will be stored

nconfs = 5 #number of conformers to create; calculation is time consuming, so here we set 5, for real tasks set 25..100
ncpu = 7 # set number of CPU cores you have

dataset_file = 'datasets/CHEMBL1075104.smi'
descriptors_folder = os.path.join('descriptors')
os.mkdir(descriptors_folder)

bags, labels, molid = calc_3d_pmapper(dataset_file, nconfs=nconfs, stereo=False, path=descriptors_folder, ncpu=ncpu)

print(f'There are {len(bags)} molecules encoded with {bags[0].shape[1]} pmapper descriptors')

In [ ]:
#Сreate a training and test set and scale the descriptors

from sklearn.model_selection import train_test_split
from miqsar.utils import  scale_descriptors

x_train, x_test, y_train, y_test, molid_train, molid_test = train_test_split(bags, labels, molid, random_state=42)
x_train, x_test = scale_descriptors(x_train, x_test)

In conventional Single-Instance QSAR, each molecule is represented by a single vector of 2D descriptors computed for the corresponding molecular graph or 3D descriptors for its lowest-energy conformation. In Multi-Instance-QSAR, a molecule is represented by a set of conformations and a set of associated vectors of descriptors which forms a bag of instances. All the considered MI algorithms can be divided into two groups – instance-based and bag-based. Instance-based algorithms consider each conformation as a separate training instance. Bag-based algorithms, on the contrary, represent a molecule by a single vector of descriptors, which is produced from the vectors of conformation descriptors. In this study, we considered several MI machine learning algorithms (see details in the original paper): *Instance-Wrapper*, *Bag-Wrapper*, *Instance-Net*, *Bag-Net*, *Bag-AttentionNet*.
<img src="img/algorithms.png" width="800"/>
*Instance-Wrapper* (a) and *Bag-Wrapper* (b) algorithms

Import MI estimators and set hyperparameters for Instance-Wrapper neural network training:

In [2]:
from miqsar.estimators.wrappers import BagWrapperMLPRegressor, InstanceWrapperMLPRegressor
from miqsar.estimators.attention_nets import AttentionNetRegressor
from miqsar.estimators.mi_nets import BagNetRegressor, InstanceNetRegressor
from sklearn.metrics import r2_score

n_epoch = 500 # number of learning epochs
batch_size = 128 # training batch size
lr = 0.001 # learning rate
weight_decay = 0.01 # L2 regularization
seed = 42
init_cuda = False # True if GPU is available for training neural network

In [ ]:
ndim = (x_train[0].shape[-1], 256, 128, 64) # the number of layers and hidden neurons in the neural network

ins_net = InstanceWrapperMLPRegressor(ndim=ndim, init_cuda=init_cuda)
ins_net.fit(x_train, y_train, n_epoch=n_epoch, batch_size=batch_size, weight_decay=weight_decay, lr=lr)

predictions = ins_net.predict(x_test)

print('3D/MI/Instance-Wrapper: r2_score test = {:.2f}'.format(r2_score(y_test, predictions)))

Also we can train MI Bag-Attention neural network. The Bag-AttentionNet provides attention weights that determine the contribution of each conformation to the predicted bioactivity. We applied regularization of attention weights (instance_dropout) to force the Bag-AttentionNet network to more strongly highlight key conformations during training. The higher the instance_dropout value (it changes from 0 to 1), the more attention weights are regularized.

In [ ]:
ndim = (x_train[0].shape[-1], 256, 128, 64) # the number of layers and hidden neurons in the main neural network
det_ndim = (64,) # the number of layers and hidden neurons in the attention neural network
instance_dropout = 0.95 # regularization of attention weights

att_net = AttentionNetRegressor(ndim=ndim, det_ndim=det_ndim, init_cuda=init_cuda)
att_net.fit(x_train, y_train, n_epoch=n_epoch, instance_dropout=instance_dropout,
            batch_size=batch_size, weight_decay=weight_decay, lr=lr)

predictions = att_net.predict(x_test)

print('3D/MI/Bag-AttentionNet: r2_score test = {:.2f}'.format(r2_score(y_test, predictions)))

A trained neural network can estimate the importance of conformations (attention weights).

In [ ]:
import seaborn as sns #also need to install seaborne

instance_weights = att_net.get_instance_weights(x_test)

n = 2 # index of molecule

sns.set(rc={'figure.figsize':(25, 8)})

x = list(range(len(instance_weights[n])))
y = instance_weights[n]

plot = sns.barplot(x=x, y=y)
plot.set(xlabel='Conformation index', ylabel='Attention weight')

Calculate descriptors for single lowest-energy conformation and train Single-Instance neural net:

In [ ]:
# calculate descriptors for single lowest-energy conformation
bags, labels, molid = calc_3d_pmapper(dataset_file, nconfs=1, stereo=False, path=descriptors_folder, ncpu=ncpu)
print(f'There are {len(bags)} molecules encoded with {bags[0].shape[1]} pmapper descriptors')

# training and test set split
x_train, x_test, y_train, y_test, molid_train, molid_test = train_test_split(bags, labels, molid, random_state=42)
x_train, x_test = scale_descriptors(x_train, x_test)

# train Single-Instance model
ndim = (x_train[0].shape[-1], 256, 128, 64) # Number of layers and hidden neurons in the neural network

ins_net = InstanceWrapperMLPRegressor(ndim=ndim, init_cuda=init_cuda)
ins_net.fit(x_train, y_train, n_epoch=n_epoch, batch_size=batch_size, weight_decay=weight_decay, lr=lr)

predictions = ins_net.predict(x_test)

print('3D/SI/Net: r2_score test = {:.2f}'.format(r2_score(y_test, predictions)))